#### <font color='orange'>Geocodificação de dados usando Latitude e Longitude</font>

##### <font color='white'>

**Explicação do Código**

1. **Importar Bibliotecas:**

- **Pandas:** para manipulação de DataFrames;
- **Requests:** para fazer solicitações HTTP;
- **Time:** para adicionar atrasos entre as solicitações.


2. **Função de Geocodificação Reversa:**

- **Reverse_geocode_nominatim:** Faz uma solicitação HTTP para a API do Nominatim usando a URL de geocodificação reversa;
- Adicionamos cabeçalhos User-Agent para se parecer com um navegador Google Chrome;
- Incluímos um tempo limite (timeout=10) e tratamos exceções de tempo limite e outras exceções de solicitação;
- A resposta da API é verificada e, se bem-sucedida, extraímos o endereço formatado dos dados retornados;
- Tqdm para a barra de progresso.


</font>

In [31]:
# Importando os pacotes necessários

import pandas as pd
import requests
import time
from tqdm import tqdm

In [38]:
# Checando a estrutura dos arquivos

df = pd.read_excel('Coordenadas.xlsx')
df.head()

,GEOCODIGO_MUNICIPIO,NOME_MUNICIPIO,LONGITUDE,LATITUDE
0,1100015,ALTA FLORESTA D'OESTE,-61.999824,-11.935540
1,1100023,ARIQUEMES,-63.033269,-9.908463
2,1100031,CABIXI,-60.544314,-13.499763
3,1100049,CACOAL,-61.442944,-11.433865
4,1100056,CEREJEIRAS,-60.818426,-13.195033


In [39]:
# Funcao para geocodificacao reversa usando API da Nominatim (OpenStreetMap)

def reverse_geocode_nomination (lat, lon):
    url = f"https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={lat}&lon={lon}"
    response = requests.get ( url, headers ={'User-Agent':'Mozilla/5.0'})
    if response.status_code == 200:
        data = response.json()
        if 'address' in data:
            address = data['address']
            formatted_address = ','.join(value for key, value in address.items())
            return formatted_address
    else:
        print(f"Error: Received response code {response.status_code}")
    return None

# Aplicar a funcaoo reverse_geocode
# Função para atualizar geolocalização com barra de progresso

def update_geolocation_with_progress(df):
    addresses = []
    for _, row in tqdm(df.iterrows(), total= df.shape[0], desc="Atualizando dados de Geolocalização"):
        address = reverse_geocode_nomination(row['LATITUDE'], row['LONGITUDE'])
        addresses.append(address)
        time.sleep(1)  # Adicionar um atraso de 1 segundo entre as solicitações
    return addresses


df['address'] = update_geolocation_with_progress(df)


Atualizando dados de Geolocalização: 100%|██████████| 299/299 [18:23<00:00,  3.69s/it]


In [40]:
df.head(5)

,GEOCODIGO_MUNICIPIO,NOME_MUNICIPIO,LONGITUDE,LATITUDE,address
0,1100015,ALTA FLORESTA D'OESTE,-61.999824,-11.935540,"Rua Goiás,Alta Floresta D'Oeste,Alta Floresta ..."
1,1100023,ARIQUEMES,-63.033269,-9.908463,"Travessa Aquariquara,Setor Institucional,Ariqu..."
2,1100031,CABIXI,-60.544314,-13.499763,"Avenida Tamoios,Cabixi,Cabixi,Região Geográfic..."
3,1100049,CACOAL,-61.442944,-11.433865,"Rua Duque de Caxias,Cristal do Arco Iris,Cacoa..."
4,1100056,CEREJEIRAS,-60.818426,-13.195033,"Rua Roraima,Cerejeiras,Cerejeiras,Região Geogr..."


In [41]:
# Exportando o novo dataframe com as informações obtidas

df.to_excel('Geocodificacao_form.xlsx', sheet_name='Coordenadas')